In [40]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

torch.set_grad_enabled(False)



In [41]:
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')


Loaded pretrained model gpt2-small into HookedTransformer


In [42]:
layer = 1
# prompt_pos = "Yes, I talk about wedding constantly"
# prompt_neg = "I do not talk about wedding constantly"
prompt_pos = "Love "
prompt_neg = "Hate"

In [43]:
logits, cache = model.run_with_cache(prompt_pos)
h_p = cache["resid_post", layer]

logits, cache = model.run_with_cache(prompt_neg)
h_n = cache["resid_post", layer]

print(h_p.shape, h_n.shape)
steering = h_p - h_n
steering.shape

torch.Size([1, 3, 768]) torch.Size([1, 3, 768])


torch.Size([1, 3, 768])

In [60]:
c = 1
def residual_stream_patching_hook(
    resid,
    hook,
    layer: int
):
    # resid shape is (batch, pos, d_model)
    len_steering = steering.shape[1]
    # print(resid.shape, steering.shape)

    
    resid[:, :len_steering, :] =  resid[:, :len_steering, :] + c * steering

    return resid

In [61]:
text = "Yesterday I said"
n_samples = 5
hook_name = utils.get_act_name("resid_post", layer)

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, layer=layer))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

Yesterday I said that ads ordered "approximately one per day." Let's clear up for one second here: 51%
Yesterday I said... even Asbus started adding credit, so I pointed out'cause that was never an excuse'
Yesterday I said/would never work in China. Life has been shorter… ~<|endoftext|>
Yesterday I said I like my taste in glass, andovenampress got a great selection at Picture Supply. This
Yesterday I said in the episode of the Video show, Cowspiracy, if you have Ultimate Magic League or R


In [58]:
steering

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.4933,  0.1339, -0.8770,  ...,  0.1296,  0.4480, -0.3008],
         [ 1.2179, -1.9815,  1.3299,  ...,  3.3314,  1.4402, -2.4186]]])

In [59]:
steering.shape

torch.Size([1, 3, 768])